In [ ]:

#Setting up the environment and import our API functions

import sys, os

# Appending notebook location
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))
sys.path.append(r'C:\Users\shail\OneDrive\Desktop\game-recommendation-engine\src\recommendation_engine.py')

from data_loader import get_app_details

print("Steam Data API Notebook Initialized!")


Steam Data API Notebook Initialized!


In [2]:
#Retrieving information for a specific Steam app using API key

api_key = "63BF59BF95391DEA2787321623A2C77E"
appid = 1245620  

# Calling get_app_details function from our data_loader
app_details = get_app_details(api_key, appid)

# Print detail;s
print(f"Details for app {appid}:")
print(app_details)



Details for app 1245620:
{'success': True, 'data': {'type': 'game', 'name': 'ELDEN RING', 'steam_appid': 1245620, 'required_age': '16', 'is_free': False, 'controller_support': 'full', 'dlc': [2778590, 2778580], 'detailed_description': '<h1>ELDEN RING Shadow of the Erdtree Edition</h1><p><img class="bb_img" src="https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/1245620/extras/ER_SHADOW-OF-THE-ERDTREE-EDITION_EN.gif?t=1739922037" /><br>ELDEN RING Shadow of the Erdtree Edition includes:<br><ul class="bb_ul"><li>ELDEN RING<br></li><li>ELDEN RING Shadow of the Erdtree expansion</li></ul></p><br><h1>ELDEN RING Shadow of the Erdtree Deluxe Edition</h1><p><img class="bb_img" src="https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/1245620/extras/ER_SHADOW-OF-THE-ERDTREE-DELUXE-EDITION_EN.gif?t=1739922037" /><br>ELDEN RING Shadow of the Erdtree Deluxe Edition includes:<br><ul class="bb_ul"><li>ELDEN RING<br></li><li>ELDEN RING Shadow of the Erdtree expansion<br><

In [3]:
#Parsing retrieved app details

if app_details and app_details.get('success', False):
    data = app_details.get('data', {})
    
    # Extracting details from the data dictionary
    name = data.get('name', 'N/A')
    app_type = data.get('type', 'N/A')
    is_free = data.get('is_free', 'N/A')
    release_date = data.get('release_date', {}).get('date', 'N/A')
    short_desc = data.get('short_description', 'N/A')
    
    # Extracting genres
    genres = data.get('genres', [])
    genres_list = [genre.get('description', '') for genre in genres] if genres else []
    
    # Printing information
    print("Name:", name)
    print("Type:", app_type)
    print("Is Free:", is_free)
    print("Release Date:", release_date)
    print("Genres:", ', '.join(genres_list) if genres_list else "N/A")
    print("Short Description:", short_desc)
else:
    print("No valid app details returned or the API call was unsuccessful.")




Name: ELDEN RING
Type: game
Is Free: False
Release Date: Feb 24, 2022
Genres: Action, RPG
Short Description: THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. Rise, Tarnished, and be guided by grace to brandish the power of the Elden Ring and become an Elden Lord in the Lands Between.


In [ ]:
#Storing details in df

import pandas as pd

# Creating a dictionary from parsed data
app_data = {
    "Name": [name],
    "Type": [app_type],
    "Is Free": [is_free],
    "Release Date": [release_date],
    "Genres": [', '.join(genres_list) if genres_list else "N/A"],
    "Short Description": [short_desc]
}

# Converting into a df
df_app_details = pd.DataFrame(app_data)

# Displaying df to inspect details
print("Aggregated App Details:")
display(df_app_details)


Aggregated App Details:


,Name,Type,Is Free,Release Date,Genres,Short Description
0,ELDEN RING,game,False,"Feb 24, 2022","Action, RPG",THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...


In [ ]:
#Aggregating details for multiple Steam apps


app_ids = [1245620, 374320, 814380]  
api_key = "63BF59BF95391DEA2787321623A2C77E"  

all_app_details = []

for appid in app_ids:
    details = get_app_details(api_key, appid)
    if details and details.get('success', False):
        data = details.get('data', {})
        app_info = {
            "Name": data.get('name', 'N/A'),
            "Type": data.get('type', 'N/A'),
            "Is Free": data.get('is_free', 'N/A'),
            "Release Date": data.get('release_date', {}).get('date', 'N/A'),
            "Genres": ', '.join([genre.get('description', '') for genre in data.get('genres', [])]) if data.get('genres') else "N/A",
            "Short Description": data.get('short_description', 'N/A')
        }
        all_app_details.append(app_info)

import pandas as pd
df_all_apps = pd.DataFrame(all_app_details)

print("Aggregated App Details for Multiple Apps:")
display(df_all_apps)


Aggregated App Details for Multiple Apps:


,Name,Type,Is Free,Release Date,Genres,Short Description
0,ELDEN RING,game,False,"Feb 24, 2022","Action, RPG",THE CRITICALLY ACCLAIMED FANTASY ACTION RPG. R...
1,DARK SOULS™ III,game,False,"Apr 11, 2016",Action,Dark Souls continues to push the boundaries wi...
2,Sekiro™: Shadows Die Twice - GOTY Edition,game,False,"Mar 21, 2019","Action, Adventure",Game of the Year - The Game Awards 2019 Best A...


In [12]:
# Search for a game by name and display matching results
from data_loader import get_steam_app_list, search_game_by_name

df_apps = get_steam_app_list()
print("Total apps fetched:", len(df_apps))

#Testing Search function
search_term = "buckshot roulette"  
results = search_game_by_name(search_term, df_apps)

print(f"Search results for '{search_term}':")
display(results[['appid', 'name']])


Total apps fetched: 235264
Search results for 'buckshot roulette':


,appid,name
212846,2835570,Buckshot Roulette


In [ ]:
#Game Selection Loop 

import pandas as pd
from data_loader import get_steam_app_list  # Ensure this is imported from your module

# Include the updated search function (if it's not already in your module, include it here)
import re

def search_game_by_name(game_name, apps_df):
    """
    Searching games with names such that the entered word appears as a separate word in the game title,
    filtering out results that contain words like 'artbook', 'soundtrack', or 'mod'.
    
    :param game_name: str, the game name.
    :param apps_df: DataFrame with at least 'appid' and 'name' columns.
    :return: DataFrame containing games that match the search term and do not include unwanted keywords.
    """
    # Builing a regex pattern for whole-word matching of the search term
    pattern = r'\b' + re.escape(game_name) + r'\b'
    matching_games = apps_df[apps_df['name'].str.contains(pattern, case=False, na=False, regex=True)]
    
    # Removing unwanted keywords
    unwanted_words = ["artbook", "soundtrack", "mod"]
    unwanted_pattern = r'\b(?:' + "|".join(unwanted_words) + r')\b'
    
    matching_games = matching_games[~matching_games['name'].str.contains(unwanted_pattern, case=False, na=False, regex=True)]
    
    return matching_games

df_apps = get_steam_app_list()
print("Total apps fetched:", len(df_apps))

# Creating list to hold the selected games
selected_games = []

while True:
    # Asking user for a game name 
    search_term = input("Enter a game name to search (or type 'done' to finish): ")
    if search_term.lower() == 'done':
        break

    # Searching for matching games
    results = search_game_by_name(search_term, df_apps)

    if results.empty:
        print(f"No matches found for '{search_term}'. Please try again.")
        continue

    # Displaying search results
    print(f"Search results for '{search_term}':")
    display(results[['appid', 'name']])
    
    selected_appid = input("Enter the appid of the most relevant game from the above results: ")

    try:
        selected_appid = int(selected_appid)
    except ValueError:
        print("Invalid appid. Please enter a numeric appid.")
        continue

    if selected_appid not in results['appid'].values:
        print(f"Appid {selected_appid} is not in the search results. Please try again.")
        continue

    selected_game = results[results['appid'] == selected_appid].iloc[0].to_dict()
    selected_games.append(selected_game)
    print(f"Game '{selected_game['name']}' added.\n")

# Converting the list of selected games into a DataFrame
df_selected = pd.DataFrame(selected_games)
print("Final selected games:")
display(df_selected)


Total apps fetched: 235232
Search results for 'elden ring':


,appid,name
98411,1922350,ELDEN RING - EU Pre-Purchase DLC Bundle
118207,1799420,ELDEN RING Bonus Gesture
165818,2855530,ELDEN RING Shadow of the Erdtree Bonus Gesture
230999,1245620,ELDEN RING
233430,2622380,ELDEN RING NIGHTREIGN


Game 'ELDEN RING' added.

Search results for 'sekiro':


,appid,name
183949,814380,Sekiro™: Shadows Die Twice


Game 'Sekiro™: Shadows Die Twice' added.

Search results for 'dark souls':


,appid,name
29978,374320,DARK SOULS™ III
30010,287360,Dark Souls II - Pre-Order DLC JP
30191,284450,DARK SOULS™ II - Season Pass
30785,271940,Dark Souls II - Pre-Order DLC ROW
30786,271941,Dark Souls II - Digital Extras
30787,271942,Dark Souls™ II Crown of the Sunken King
30788,271943,DARK SOULS™ II Crown of the Old Iron King
32434,236430,DARK SOULS™ II
35258,81964,Dark Souls PTDE Release Trailer
35345,81951,Dark Souls GamesCom Trailer PEGI


Game 'DARK SOULS™ II' added.

No matches found for 'armored core 6'. Please try again.
Search results for 'armored core':


,appid,name
231198,1888160,ARMORED CORE™ VI FIRES OF RUBICON™


Game 'ARMORED CORE™ VI FIRES OF RUBICON™' added.

Final selected games:


,appid,name
0,1245620,ELDEN RING
1,814380,Sekiro™: Shadows Die Twice
2,236430,DARK SOULS™ II
3,1888160,ARMORED CORE™ VI FIRES OF RUBICON™


In [ ]:
#Adding Reviews column to selected games df
from data_loader import get_app_reviews
import pandas as pd

def add_reviews_to_df(df):
    """
    For each game in the DataFrame, fetch the reviews and add them in a new column.
    
    :param df: DataFrame containing at least an 'appid' column.
    :return: DataFrame with a new 'Reviews' column.
    """
    reviews_list = []
    for index, row in df.iterrows():
        appid = row['appid']
        print(f"Fetching reviews for appid: {appid}...")
        reviews_text = get_app_reviews(appid, num_reviews=20)
        reviews_list.append(reviews_text)
    df['Reviews'] = reviews_list
    return df

df_selected = add_reviews_to_df(df_selected)
print("Updated DataFrame with Reviews:")
display(df_selected)


Fetching reviews for appid: 1245620...
Fetching reviews for appid: 814380...
Fetching reviews for appid: 236430...
Fetching reviews for appid: 1888160...
Updated DataFrame with Reviews:


,appid,name,Reviews
0,1245620,ELDEN RING,"mad difficult Elden mid , Reddit souls , its v..."
1,814380,Sekiro™: Shadows Die Twice,best soulsborne game by a long shot Sim great ...
2,236430,DARK SOULS™ II,O jogo é injogável? Não. Mas ele faz MUITA for...
3,1888160,ARMORED CORE™ VI FIRES OF RUBICON™,Ride wife . life good\nWife fight back \nKill ...


In [9]:
# Save as CSV
df_selected.to_csv('selected_games.csv', index=False)

"""# Or save as a pickle (which preserves data types)
df_selected.to_pickle('selected_games.pkl')"""


"# Or save as a pickle (which preserves data types)\ndf_selected.to_pickle('selected_games.pkl')"

In [10]:
# Cell: Fetch and store a larger dataset of Steam games with reviews
from data_loader import fetch_game_reviews

# Your Steam API key (replace this with your actual key)
api_key = "63BF59BF95391DEA2787321623A2C77E"

# Fetch data for 100 games (adjust number as needed)
df_all_games = fetch_game_reviews(api_key, num_games=1000, num_reviews=50)

# Save the expanded dataset for use in Notebook 2
df_all_games.to_csv("all_steam_games_with_reviews.csv", index=False)

print("Dataset saved successfully!")
display(df_all_games.head())


Total games available on Steam: 235232
Fetching reviews for WildStar: Cute & Cuddly Pack (appid: 479061)...
Fetching reviews for Train Simulator: Salzburg - Rosenheim Route Add-On (appid: 1804514)...
Fetching reviews for Funko Fusion - KFC Chef Colonel (appid: 3112580)...
No reviews found for appid 3112580
Fetching reviews for They Bleed Pixels Trailer (appid: 81952)...
No reviews found for appid 81952
Fetching reviews for Pirates Never Die (appid: 3277140)...
Fetching reviews for Come On Down (appid: 425970)...
Fetching reviews for Monster Knockout Idle DLC (appid: 2258610)...
Fetching reviews for The Parish Demo (appid: 3213460)...
Fetching reviews for Train Sim World® 4 Compatible: DB BR 204 Add-On (appid: 2375901)...
Fetching reviews for Girl & Gem Magic (appid: 2122070)...
Fetching reviews for Marie's Room - Anniversary Soundtrack (appid: 1971600)...
Fetching reviews for Law Mower (appid: 660940)...
Fetching reviews for Fabergegg (appid: 1177540)...
Fetching reviews for Niknaks (a

,appid,name,Reviews
90770,479061,WildStar: Cute & Cuddly Pack,DO NOT BUY!\n\nBought this pack and only got o...
117809,1804514,Train Simulator: Salzburg - Rosenheim Route Ad...,"Ich finde die Strecke gut, eine schöne Erweite..."
192055,3277140,Pirates Never Die,This is the only game that I have ever asked S...
26025,425970,Come On Down,Come on Down is a stereotypical movie that cre...
113143,2258610,Monster Knockout Idle DLC,2/10 ok??? why is this part of an angry birds ...
